<a href="https://colab.research.google.com/github/jpordoy/RSNA-2024-Lumbar-Spine-Degenerative-Classification/blob/main/SSL_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [215]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.9 MB/s eta 0:00:00


In [3]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'rsna-2024-lumbar-spine-degenerative-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71549%2F8561470%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240713%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240713T154049Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D16456db4721f570942963a87a3092d4a28bc37fd425f94d167b47db4c424a73785e36e403d4225b8f632b9890e0b26e5aa9b44597ada4f70df06d6ed12870c765d1bab8e39571f3755c81533a451e9f5b0e478ccaef9db158e80af3840bdc25c70e4a414c03f10306309dca915dc1cafe2748cd0f2eee8d538aae0be1e45a96c599857c161562b59901ae009b454a5afb90c10e0da1c63e07c899a36e8fc6e6ec856b93002cb3b7839feff7e9b5450123286a2fb1d84c0190cfb0e7b5b61bf980ba971eb2cd18de891b040785bf12f91e8baf66745d8370edaf514f8354addabbf15525ceafc773f290d07bcfc66b2fd0ec2d6a3f71434daf49e42f102d426df'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 30303207710 bytes downloaded
Downloaded and uncompressed: rsna-2024-lumbar-spine-degenerative-classification
Data source import complete.


In [4]:
import pandas as pd
import numpy as np
import pydicom
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [33]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image as PILImage
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import glob

df_train = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
df_train_desc = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
df_train_desc['image_path'] = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/" + df_train_desc['study_id'].astype(str) +"/"+ df_train_desc['series_id'].astype(str) + "/"+ df_train_desc['instance_number'].astype(str) + ".dcm"
df_train_desc.sample(5)


,study_id,series_id,instance_number,condition,level,x,y,image_path
32851,2905025904,282288462,17,Left Neural Foraminal Narrowing,L4/L5,313.670330,456.307692,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
5358,462494704,414145483,9,Spinal Canal Stenosis,L1/L2,288.149123,195.087719,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
47276,4164326469,3518664032,11,Spinal Canal Stenosis,L1/L2,221.285199,147.465704,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
19388,1742195308,3210956062,2,Left Subarticular Stenosis,L1/L2,209.382458,218.102719,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
6579,607343607,918340269,21,Right Subarticular Stenosis,L4/L5,230.422920,250.412005,/kaggle/input/rsna-2024-lumbar-spine-degenerat...


In [40]:
# Directory containing all images
image_dir = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/"

# Get all image paths in the directory
all_image_paths = glob.glob(os.path.join(image_dir, '**', '*.dcm'), recursive=True)

# Convert to a set for faster lookup
annotated_image_paths = set(df_train_desc['image_path'].tolist())

# Identify unannotated images
unannotated_image_paths = [path for path in all_image_paths if path not in annotated_image_paths]

print(f"Total annotated images: {len(annotated_image_paths)}")
print(f"Total unannotated images: {len(unannotated_image_paths)}")

Total annotated images: 24546
Total unannotated images: 122672


In [32]:
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='value')

# Split the 'condition_level' column to extract 'condition' and 'level'
df_train_melted[['conditions', 'level']] = df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:, 1:]
df_train_melted['condition'] = df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[:-2])).str.replace("_"," ").str.title()
df_train_melted['level'] = df_train_melted['conditions'].str.upper() +"/"+ df_train_melted['level'].str.upper()
# Remove the original 'condition_level' column
df_train_melted = df_train_melted.drop(columns=['condition_level', 'conditions'])

# Rename columns for clarity
df_train_melted = df_train_melted.rename(columns={'value': 'severity'})
df_train_melted.sample(5)

,study_id,severity,level,condition
4900,2109299850,Normal/Mild,L3/L4,Spinal Canal Stenosis
13376,3318343342,Normal/Mild,L2/L3,Left Neural Foraminal Narrowing
5976,109454808,Normal/Mild,L4/L5,Spinal Canal Stenosis
37383,3971523744,Moderate,L4/L5,Left Subarticular Stenosis
37843,701848105,Severe,L5/S1,Left Subarticular Stenosis


In [34]:
import pandas as pd

# Load the CSV files
df_train = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
df_train_desc = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")

# Add the image path to the descriptive DataFrame
df_train_desc['image_path'] = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/" + df_train_desc['study_id'].astype(str) + "/" + df_train_desc['series_id'].astype(str) + "/" + df_train_desc['instance_number'].astype(str) + ".dcm"

# Reshape the training DataFrame
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='severity')

# Extract 'condition' and 'level' from 'condition_level'
df_train_melted[['conditions', 'level']] = df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:, 1:]
df_train_melted['condition'] = df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[:-2])).str.replace("_"," ").str.title()
df_train_melted['level'] = df_train_melted['conditions'].str.upper() + "/" + df_train_melted['level'].str.upper()

# Drop unnecessary columns
df_train_melted = df_train_melted.drop(columns=['condition_level', 'conditions'])

# Merge the DataFrames on 'study_id', 'level', and 'condition'
df_final = pd.merge(df_train_desc, df_train_melted, on=['study_id', 'level', 'condition'], how='inner')

# Display a sample of the final DataFrame
df_final.sample(5)


,study_id,series_id,instance_number,condition,level,x,y,image_path,severity
12568,1119763924,1930073893,6,Right Neural Foraminal Narrowing,L2/L3,272.856940,180.106874,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
38815,3427409235,520744343,6,Right Neural Foraminal Narrowing,L1/L2,224.156889,85.653479,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
30012,2662989538,1211513177,19,Left Subarticular Stenosis,L4/L5,134.435772,129.665041,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Moderate
25797,2325650566,2076691051,12,Left Neural Foraminal Narrowing,L4/L5,297.247349,401.691420,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Moderate
17301,1544019340,2670611489,13,Right Subarticular Stenosis,L4/L5,122.424467,114.120482,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Moderate


In [37]:
# Select a random row from df_final
random_row_final = df_final.sample(n=1)

# Extract the values to find corresponding rows in df_train_desc and df_train_melted
study_id = random_row_final.iloc[0]['study_id']
series_id = random_row_final.iloc[0]['series_id']
level = random_row_final.iloc[0]['level']
condition = random_row_final.iloc[0]['condition']

# Select corresponding rows from df_train_desc and df_train_melted
row_desc = df_train_desc[
    (df_train_desc['study_id'] == study_id) &
    (df_train_desc['series_id'] == series_id) &
    (df_train_desc['level'] == level) &
    (df_train_desc['condition'] == condition)
]

row_melted = df_train_melted[
    (df_train_melted['study_id'] == study_id) &
    (df_train_melted['level'] == level) &
    (df_train_melted['condition'] == condition)
]

# Print the rows to verify the merge
print("Random row from df_final:")
random_row_final.head()

Random row from df_final:


,study_id,series_id,instance_number,condition,level,x,y,image_path,severity
13089,1166421944,4052001585,8,Left Subarticular Stenosis,L2/L3,140.437321,118.097421,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild


In [38]:
print("\nCorresponding row from df_train_desc:")
row_desc.head()


Corresponding row from df_train_desc:


,study_id,series_id,instance_number,condition,level,x,y,image_path
13089,1166421944,4052001585,8,Left Subarticular Stenosis,L2/L3,140.437321,118.097421,/kaggle/input/rsna-2024-lumbar-spine-degenerat...


In [46]:
print("\nCorresponding row from df_train_melted:")
row_melted.head()


Corresponding row from df_train_melted:


,study_id,severity,level,condition
32129,1166421944,Normal/Mild,L2/L3,Left Subarticular Stenosis


In [47]:
from sklearn.model_selection import train_test_split

# Split df_final into training and validation sets
df_train_labeled, df_val_labeled = train_test_split(df_final, test_size=0.2, random_state=333)

print(f"Total labeled training images: {len(df_train_labeled)}")
print(f"Total labeled validation images: {len(df_val_labeled)}")


Total labeled training images: 38953
Total labeled validation images: 9739


In [49]:
labels_list = list(set(df_train_labeled['severity'].values))
label2id, id2label = dict(), dict()
for i, label in enumerate(labels_list):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)


{0: nan, 1: 'Normal/Mild', 2: 'Severe', 3: 'Moderate'} 

 {nan: 0, 'Normal/Mild': 1, 'Severe': 2, 'Moderate': 3}


In [50]:
import numpy as np
import pydicom
from PIL import Image as PILImage

def load_and_preprocess_image(file_path, target_shape=(128, 128)):
    dicom = pydicom.dcmread(file_path)
    image = dicom.pixel_array
    image = (image / np.max(image) * 255).astype(np.uint8)
    pil_image = PILImage.fromarray(image)
    resized_image = pil_image.resize(target_shape)
    return np.array(resized_image)


In [59]:
import numpy as np
import pydicom
from PIL import Image as PILImage
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=32, target_shape=(128, 128), shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_shape = target_shape
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        image_paths_temp = [self.image_paths[k] for k in indexes]
        labels_temp = [self.labels[k] for k in indexes]
        X, y = self.__data_generation(image_paths_temp, labels_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, labels_temp):
        X = np.empty((self.batch_size, *self.target_shape, 3))  # Ensure all images have 3 channels
        y = np.empty((self.batch_size), dtype=int)
        for i, (img_path, label) in enumerate(zip(image_paths_temp, labels_temp)):
            img = pydicom.dcmread(img_path).pixel_array
            img = (img / np.max(img) * 255).astype(np.uint8)
            img = PILImage.fromarray(img).resize(self.target_shape)
            img = np.array(img)
            if img.ndim == 2:  # Convert grayscale to RGB
                img = np.stack((img,)*3, axis=-1)
            img = img / 255.0
            X[i,] = img
            y[i] = label
        return X, y

# Prepare the data
train_data = {
    'image_path': df_train_labeled['image_path'].values,
    'label': df_train_labeled['severity'].map(label2id).values
}

val_data = {
    'image_path': df_val_labeled['image_path'].values,
    'label': df_val_labeled['severity'].map(label2id).values
}

# Define the generators
train_generator = DataGenerator(train_data['image_path'], train_data['label'])
val_generator = DataGenerator(val_data['image_path'], val_data['label'])


In [60]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define the model using EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(label2id), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [63]:
# Define batch size
batch_size = 128  # Adjust based on your system's capacity and performance

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    batch_size=batch_size
)

Epoch 1/5
1217/1217 [==============================] - 487s 401ms/step - loss: 0.4711 - accuracy: 0.8058 - val_loss: 0.6242 - val_accuracy: 0.7737
Epoch 2/5
1217/1217 [==============================] - 489s 402ms/step - loss: 0.4538 - accuracy: 0.8092 - val_loss: 1.7395 - val_accuracy: 0.0143
Epoch 3/5
1217/1217 [==============================] - 487s 400ms/step - loss: 0.4381 - accuracy: 0.8151 - val_loss: 0.6948 - val_accuracy: 0.7734
Epoch 4/5
1217/1217 [==============================] - 486s 400ms/step - loss: 0.4165 - accuracy: 0.8206 - val_loss: 0.7541 - val_accuracy: 0.7735
Epoch 5/5
1217/1217 [==============================] - 496s 408ms/step - loss: 0.3962 - accuracy: 0.8247 - val_loss: 1.4775 - val_accuracy: 0.2441


In [67]:
from sklearn.metrics import classification_report, confusion_matrix

# Calculate total number of validation samples
total_val_samples = len(val_generator.labels)

# Generate predictions for the validation set
val_predictions = model.predict(val_generator, steps=len(val_generator), verbose=1)
val_predictions = np.argmax(val_predictions, axis=1)

# Ensure the length of predictions matches the total number of samples
if len(val_predictions) != total_val_samples:
    # Adjusting for potential batch size differences
    if len(val_predictions) < total_val_samples:
        # If predictions are shorter, it could be due to the last batch having fewer samples
        total_val_samples = len(val_predictions)
    else:
        # If predictions are longer, there might be an issue with how batches are processed
        raise ValueError(f"Number of predictions ({len(val_predictions)}) does not match number of validation samples ({total_val_samples})")

# Ensure target_names are strings
target_names = list(map(str, label2id.keys()))

# Print classification report and confusion matrix
print(classification_report(val_generator.labels[:total_val_samples], val_predictions, target_names=target_names))
print(confusion_matrix(val_generator.labels[:total_val_samples], val_predictions))


304/304 [==============================] - 98s 322ms/step
              precision    recall  f1-score   support

         nan       0.00      0.00      0.00         6
 Normal/Mild       0.78      0.26      0.39      7525
      Severe       0.06      0.45      0.11       590
    Moderate       0.17      0.13      0.14      1607

    accuracy                           0.25      9728
   macro avg       0.25      0.21      0.16      9728
weighted avg       0.63      0.25      0.34      9728

[[   0    1    4    1]
 [1415 1985 3182  943]
 [ 117  139  268   66]
 [ 293  426  684  204]]


In [210]:
import numpy as np
import pydicom
from PIL import Image as PILImage
from tensorflow.keras.utils import Sequence

class SSLDataGenerator(Sequence):
    def __init__(self, image_paths_labeled, labels_labeled=None, image_paths_unlabeled=None, batch_size=32, target_shape=(128, 128)):
        self.image_paths_labeled = image_paths_labeled
        self.labels_labeled = labels_labeled
        self.image_paths_unlabeled = image_paths_unlabeled
        self.batch_size = batch_size
        self.target_shape = target_shape
        self.indices = np.arange(len(image_paths_labeled))

        if self.image_paths_unlabeled is not None:
            self.indices_unlabeled = np.arange(len(image_paths_unlabeled))

    def __len__(self):
        return int(np.ceil(len(self.image_paths_labeled) / float(self.batch_size)))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labeled = self.__data_generation(self.image_paths_labeled[indices], self.labels_labeled[indices])

        if self.image_paths_unlabeled is not None:
            indices_unlabeled = self.indices_unlabeled[index * self.batch_size:(index + 1) * self.batch_size]
            batch_unlabeled = self.__data_generation(self.image_paths_unlabeled[indices_unlabeled])
        else:
            batch_unlabeled = None

        return batch_labeled, batch_unlabeled

    def on_epoch_end(self):
        np.random.shuffle(self.indices)
        if self.image_paths_unlabeled is not None:
            np.random.shuffle(self.indices_unlabeled)

    def __data_generation(self, image_paths_temp, labels_temp=None):
        X = np.empty((len(image_paths_temp), *self.target_shape, 3))
        if labels_temp is not None:
            y = np.empty(len(image_paths_temp), dtype=int)
            for i, (img_path, label) in enumerate(zip(image_paths_temp, labels_temp)):
                img = pydicom.dcmread(img_path).pixel_array
                img = (img / np.max(img) * 255).astype(np.uint8)
                img = PILImage.fromarray(img).resize(self.target_shape)
                img = np.array(img)
                if img.ndim == 2:
                    img = np.stack((img,) * 3, axis=-1)
                img = img / 255.0
                X[i,] = img
                y[i] = label
            return X, y
        else:
            for i, img_path in enumerate(image_paths_temp):
                img = pydicom.dcmread(img_path).pixel_array
                img = (img / np.max(img) * 255).astype(np.uint8)
                img = PILImage.fromarray(img).resize(self.target_shape)
                img = np.array(img)
                if img.ndim == 2:
                    img = np.stack((img,) * 3, axis=-1)
                img = img / 255.0
                X[i,] = img
            return X


In [211]:
# Initialize SSL Data Generators
train_generator_labeled = DataGenerator(train_data['image_path'], train_data['label'])
train_generator_unlabeled = DataGenerator(unannotated_image_paths, batch_size=32)

In [212]:
# Define SSL model architecture for three-class classification
def create_ssl_model(input_shape=(128, 128, 3), num_classes=3):
    # Shared convolutional base
    base_model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten()
    ])

    # Labeled data input
    input_labeled = Input(shape=input_shape, name='input_labeled')
    x_labeled = base_model(input_labeled)
    output_labeled = Dense(num_classes, activation='softmax', name='output_labeled')(x_labeled)

    # Unlabeled data input
    input_unlabeled = Input(shape=input_shape, name='input_unlabeled')
    x_unlabeled = base_model(input_unlabeled)
    output_unlabeled = Dense(num_classes, activation='softmax', name='output_unlabeled')(x_unlabeled)

    # SSL model
    ssl_model = Model(inputs=[input_labeled, input_unlabeled], outputs=[output_labeled, output_unlabeled])

    return ssl_model

# Create SSL model for three-class classification
ssl_model = create_ssl_model(num_classes=3)


In [213]:
# Compile SSL model
ssl_model.compile(optimizer='adam',
                  loss={'output_labeled': 'sparse_categorical_crossentropy', 'output_unlabeled': 'categorical_crossentropy'},
                  metrics=['accuracy'])


In [214]:
num_epochs_ssl = 10

train_generator_ssl = SSLDataGenerator(train_data_labeled['image_path'], train_data_labeled['label'],
                                       image_paths_unlabeled=train_data_unlabeled, # Remove the list() call and the 'image_path' key
                                       batch_size=32)
for epoch in range(num_epochs_ssl):
    print(f"Epoch {epoch+1}/{num_epochs_ssl}")

    for batch_labeled, batch_unlabeled in train_generator_ssl:
        # Train the SSL model on labeled and potentially unlabeled data
        ssl_model.train_on_batch({'input_labeled': batch_labeled[0], 'input_unlabeled': batch_unlabeled[0]},
                                 {'output_labeled': batch_labeled[1], 'output_unlabeled': batch_unlabeled[1]})

        # Optional: Log or print metrics for monitoring
        labeled_loss = ssl_model.evaluate(batch_labeled[0], batch_labeled[1], verbose=0)
        unlabeled_loss = ssl_model.evaluate(batch_unlabeled[0], batch_unlabeled[1], verbose=0) if batch_unlabeled is not None else None

        print(f"  Batch - Labeled Loss: {labeled_loss[0]}, Unlabeled Loss: {unlabeled_loss[0] if unlabeled_loss else None}")

print("Training completed.")


Epoch 1/10


TypeError: only integer scalar arrays can be converted to a scalar index